In [1]:
import pandas as pd
from sklearn.datasets import make_regression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import geopy.distance
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import *

In [2]:
train_file = 'data/out/file_cleaned.csv'
test_file = 'data/in/test.csv'

train_data = pd.read_csv(train_file)
train_data.drop(columns=['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace=True)
test_data = pd.read_csv(test_file)

In [3]:
train_data.head()

,id,fecha,latitud_origen,longitud_origen,latitud_destino,longitud_destino,distancia,tiempo,region_origen,region_destino,distancia_km,distancia_api,tiempo_api
0,124092420,2021-07-28,-11.965070,-77.12269,-12.07020,-77.13524,14747.0,1775.0,Callao,Lima,14.747,2449.4,281.1
1,124091219,2021-07-25,-12.155090,-76.97996,-12.07570,-76.91754,17171.0,1821.0,Lima,Lima,17.171,9842.2,1145.1
2,124091187,2021-07-25,-14.533770,-75.18460,-14.88630,-74.99414,58708.0,3842.0,Ica,Ica,58.708,44813.5,4059.7
3,124093495,2021-08-01,-5.201517,-80.64635,-5.38453,-80.62314,23611.0,1446.0,Departamento de Piura,Departamento de Piura,23.611,3328.1,326.1
4,124092156,2021-07-27,-11.967470,-77.12508,-12.03500,-77.07444,14229.0,1974.0,Callao,Lima,14.229,7593.7,749.9


In [ ]:
test_data.head()

In [ ]:
X=train_data.iloc[:,[False,False,True,True,True,True,False,False,True,True,True,True,True]].values
y=train_data.iloc[:,6:8].values
yhat=test_data.iloc[:,2:].values

In [ ]:
from sklearn import preprocessing
preprocessing.minmax_scale(X)
preprocessing.minmax_scale(yhat)

In [ ]:
from keras.optimizers import Adam

def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
    model.compile(loss='mape', optimizer='adam')
    return model

In [ ]:
n_inputs, n_outputs = X.shape[1], y.shape[1]
print(X.shape[1], y.shape[1])

In [ ]:
from keras.callbacks import ModelCheckpoint

def get_callbacks():
    filepath = 'data/out/try5.h5'
    callback1 = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callback2 = CSVLogger('data/out/try5.csv')
    
    return [callback1,callback2]

In [ ]:
model = get_model(n_inputs, n_outputs)
history = model.fit(X, y, verbose=1, epochs=300, validation_split=0.2, callbacks=get_callbacks())

In [ ]:
#Cargando el mejor modelo

from keras.models import load_model
best_model_tf = load_model('data/out/try5.h5')

ypred = best_model_tf.predict(yhat)

In [ ]:
print(ypred)

In [ ]:
best_model_tf.evaluate(X,y)

In [ ]:
ids = pd.read_csv(test_file)
ids_1 = ids['ID']

In [ ]:
df = pd.DataFrame()
df['ID'] = ids_1
df['DISTANCIA'] = ypred[:,0]
df['TIEMPO'] = ypred[:,1]
print(df)

In [ ]:
df.to_csv('data/out/try5_out.csv',index=False)

In [ ]:
#Graficar Epochs

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(0,len(loss))
fig = plt.gcf()
fig.set_size_inches(16, 8)

fig = plt.gcf()
fig.set_size_inches(16, 8)
plt.plot(epochs, loss, 'r', label='Training Loss',marker = "o")
plt.plot(epochs, val_loss, 'b', label='Validation Loss',marker = "o")
plt.title('Training and validation Loss')
plt.legend(loc=0)
#plt.savefig('Multiclass Model .png')
plt.figure()
plt.show()